In [1]:
import urllib.request, json
import folium
import urllib.request, json
from datetime import datetime as dt
import time

import pandas as pd

#Weather office codes for api.weather.gov, extracted from - https://api.weather.gov/offices/RFC
weather_office = '\"AKQ\",\"ALY\",\"BGM\",\"BOX\",\"BTV\",\"BUF\",\"CAE\",\"CAR\",\"CHS\",\"CLE\",\"CTP\",\"GSP\",\"GYX\",\"ILM\",\"ILN\",\"LWX\",\"MHX\",\"OKX\",\"PBZ\",\"PHI\",\"RAH\",\"RLX\",\"RNK\",\"ABQ\",\"AMA\",\"BMX\",\"BRO\",\"CRP\",\"EPZ\",\"EWX\",\"FFC\",\"FWD\",\"HGX\",\"HUN\",\"JAN\",\"JAX\",\"KEY\",\"LCH\",\"LIX\",\"LUB\",\"LZK\",\"MAF\",\"MEG\",\"MFL\",\"MLB\",\"MOB\",\"MRX\",\"OHX\",\"OUN\",\"SHV\",\"SJT\",\"SJU\",\"TAE\",\"TBW\",\"TSA\",\"ABR\",\"APX\",\"ARX\",\"BIS\",\"BOU\",\"CYS\",\"DDC\",\"DLH\",\"DMX\",\"DTX\",\"DVN\",\"EAX\",\"FGF\",\"FSD\",\"GID\",\"GJT\",\"GLD\",\"GRB\",\"GRR\",\"ICT\",\"ILX\",\"IND\",\"IWX\",\"JKL\",\"LBF\",\"LMK\",\"LOT\",\"LSX\",\"MKX\",\"MPX\",\"MQT\",\"OAX\",\"PAH\",\"PUB\",\"RIW\",\"SGF\",\"TOP\",\"UNR\",\"BOI\",\"BYZ\",\"EKA\",\"FGZ\",\"GGW\",\"HNX\",\"LKN\",\"LOX\",\"MFR\",\"MSO\",\"MTR\",\"OTX\",\"PDT\",\"PIH\",\"PQR\",\"PSR\",\"REV\",\"SEW\",\"SGX\",\"SLC\",\"STO\",\"TFX\",\"TWC\",\"VEF\",\"AER\",\"AFC\",\"AFG\",\"AJK\",\"ALU\",\"GUM\",\"HPA\",\"HFO\",\"PPG\",\"STU\",\"NH1\",\"NH2\",\"ONA\",\"ONP\"'
weather_office = weather_office.replace('"', '').split(",")

def return_json(url_api):
    with urllib.request.urlopen(url_api) as url:
        data = json.load(url)
        return data
def weather_api_coordinates(lat, long):
    url_api = "https://api.weather.gov/points/" + str(lat) + ',' + str(long)
    return(return_json(url_api))

def polygon_to_map(coordinates, invert = False): #coordinates in the format of [[lat, lon]...]
    if invert:
        coordinates = [item[::-1] for item in coordinates]
        
    m = folium.Map(location=coordinates[0], zoom_start=13)
    folium.Polygon(
        locations = coordinates,  # Flip lon/lat to lat/lon
        color='blue',
        fill=True,
        fill_color='cyan',
        fill_opacity=0.5
    ).add_to(m)
    return(m)

api_weather_url_base = "https://api.weather.gov/points/"
api_weather_url = api_weather_url_base + "47.675908,-122.208008"
#api_weather_url = api_weather_url_base + "46.189070,-122.184252"
start = time.time()
data = return_json(api_weather_url)
print(time.time()-start)
forecast = return_json(data['properties']['forecast'])
print(time.time()-start)

print(api_weather_url)

print(data['properties']['forecast'])
print(data['properties']['forecastHourly'])
print(data['properties']['forecastGridData'])
print(data['properties']['observationStations'])

5.067967176437378
7.063474178314209
https://api.weather.gov/points/47.675908,-122.208008
https://api.weather.gov/gridpoints/SEW/129,70/forecast
https://api.weather.gov/gridpoints/SEW/129,70/forecast/hourly
https://api.weather.gov/gridpoints/SEW/129,70
https://api.weather.gov/gridpoints/SEW/129,70/stations


In [134]:
temp = return_json(data['properties']['forecastGridData'])
data['properties']['forecastGridData']
forecastGridData_detailed = temp.copy()

del forecastGridData_detailed['@context']
del forecastGridData_detailed['id']
del forecastGridData_detailed['type']
del forecastGridData_detailed['geometry']
del forecastGridData_detailed['properties']['@id']
del forecastGridData_detailed['properties']['@type']
del forecastGridData_detailed['properties']['elevation']
del forecastGridData_detailed['properties']['forecastOffice']
del forecastGridData_detailed['properties']['gridId']
del forecastGridData_detailed['properties']['gridX']
del forecastGridData_detailed['properties']['gridY']
del forecastGridData_detailed['properties']['updateTime']
del forecastGridData_detailed['properties']['validTimes']
del forecastGridData_detailed['properties']
forecastGridData_detailed

{}

In [122]:
forecastGridData_detailed['properties']

{'temperature': {'uom': 'wmoUnit:degC',
  'values': [{'validTime': '2024-12-13T04:00:00+00:00/PT6H',
    'value': 6.666666666666667},
   {'validTime': '2024-12-13T10:00:00+00:00/PT2H', 'value': 6.111111111111111},
   {'validTime': '2024-12-13T12:00:00+00:00/PT6H', 'value': 5.555555555555555},
   {'validTime': '2024-12-13T18:00:00+00:00/PT1H', 'value': 6.111111111111111},
   {'validTime': '2024-12-13T19:00:00+00:00/PT1H', 'value': 6.666666666666667},
   {'validTime': '2024-12-13T20:00:00+00:00/PT1H', 'value': 7.222222222222222},
   {'validTime': '2024-12-13T21:00:00+00:00/PT3H', 'value': 8.333333333333334},
   {'validTime': '2024-12-14T00:00:00+00:00/PT1H', 'value': 8.88888888888889},
   {'validTime': '2024-12-14T01:00:00+00:00/PT3H', 'value': 8.333333333333334},
   {'validTime': '2024-12-14T04:00:00+00:00/PT10H',
    'value': 7.777777777777778},
   {'validTime': '2024-12-14T14:00:00+00:00/PT1H', 'value': 7.222222222222222},
   {'validTime': '2024-12-14T15:00:00+00:00/PT4H', 'value': 8.

In [102]:
df_dewpoint = pd.DataFrame(forecastGridData_detailed['properties']['dewpoint']['values'])
df_dewpoint['units_dewpoint'] = forecastGridData_detailed['properties']['dewpoint']['uom']
df_dewpoint['validTime'] = pd.to_datetime(df_dewpoint['validTime'].apply(lambda x: x.split('/')[0]))
df_dewpoint['validTime'] = df_dewpoint['validTime'].apply(lambda x: str(x))
df_dewpoint = df_dewpoint.set_index('validTime')
df_dewpoint.rename(columns={'value' : 'dewpoint_value'}, inplace=True)

df_temp = pd.DataFrame(forecastGridData_detailed['properties']['temperature']['values'])
df_temp['units_temp'] = forecastGridData_detailed['properties']['temperature']['uom']
df_temp['validTime'] = pd.to_datetime(df_temp['validTime'].apply(lambda x: x.split('/')[0]))
df_temp['validTime'] = df_temp['validTime'].apply(lambda x: str(x))
df_temp = df_temp.set_index('validTime')
#df_dewpoint
pd.concat((df_dewpoint, df_temp), axis = 1 )
#pd.concat((pd.DataFrame(), df_temp), axis = 1 )
#df_temp

,dewpoint_value,units_dewpoint,value,units_temp
validTime,,,,
2024-12-13 04:00:00+00:00,5.555556,wmoUnit:degC,6.666667,wmoUnit:degC
2024-12-13 08:00:00+00:00,5.000000,wmoUnit:degC,NaN,NaN
2024-12-13 09:00:00+00:00,5.555556,wmoUnit:degC,NaN,NaN
2024-12-13 10:00:00+00:00,5.000000,wmoUnit:degC,6.111111,wmoUnit:degC
2024-12-13 12:00:00+00:00,4.444444,wmoUnit:degC,5.555556,wmoUnit:degC
...,...,...,...,...
2024-12-19 22:00:00+00:00,NaN,NaN,11.111111,wmoUnit:degC
2024-12-20 00:00:00+00:00,NaN,NaN,10.555556,wmoUnit:degC
2024-12-20 08:00:00+00:00,NaN,NaN,7.222222,wmoUnit:degC


In [133]:
df_temporary = pd.DataFrame()
df_full = pd.DataFrame()
categories = {'temperature': [],
              'humidity' : [],
              'wind' : [],
              'precipitation' : [],
              'snow' : []}
for k in forecastGridData_detailed['properties'].keys():
    variable = forecastGridData_detailed['properties'][k]
    if len(variable['values']) == 0:
        continue
    df_temporary = pd.DataFrame(variable['values']).rename(columns={'value': k +"_value"})
    if 'uom' in variable:
        df_temporary[k+"_units"] = variable['uom'].split(':')[-1]

    df_temporary['validTime'] = pd.to_datetime(df_temporary['validTime'].apply(lambda x: x.split('/')[0]))
    #df_temporary['validTime'] = df_temporary['validTime'].apply(lambda x: str(x))
    df_temporary = df_temporary.set_index('validTime')
    df_full = pd.concat((df_full, df_temporary), axis = 1)
    found = 0
    for c in categories:
        if c in k.lower():
            categories[c].append(k)
            found = 1
    if not found:
        print(k)


#df_temporary
#df_full
#forecastGridData_detailed['properties']['temperature']['values']
#categories
#dk

dewpoint
heatIndex
skyCover
weather
hazards
iceAccumulation
mixingHeight
hainesIndex
lightningActivityLevel
waveHeight
wavePeriod2
probabilityOfThunder


In [125]:
'A'.lower()

'a'

In [123]:
temp = return_json(data['properties']['forecastHourly'])

forecastHourly_detailed = temp.copy()

del forecastHourly_detailed['@context']
del forecastHourly_detailed['type']
del forecastHourly_detailed['geometry']
del forecastHourly_detailed['properties']['units']
del forecastHourly_detailed['properties']['forecastGenerator']
del forecastHourly_detailed['properties']['generatedAt']
del forecastHourly_detailed['properties']['updateTime']
del forecastHourly_detailed['properties']['validTimes']

df = pd.DataFrame(forecastHourly_detailed['properties']['periods'])
df.drop(['number', 'name', 'endTime', 'temperatureTrend', 'detailedForecast'], axis = 1, inplace = True)
df['startTime'] = pd.to_datetime(df['startTime'])
df['probabilityOfPrecipitation'] = df['probabilityOfPrecipitation'].apply( lambda x: x['value'] / 100 if x['value'] is not None else 0 )


df['startTime'].apply(lambda x : x.strftime('%Y-%m-%d %I:%M:%S %p'))
df#df['startTime']
#del forecastHourly_detailed['properties']['periods']
#forecastHourly_detailed

,startTime,isDaytime,temperature,temperatureUnit,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast
0,2024-12-13 08:00:00-08:00,True,42,F,0.22,"{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 90}",5 mph,ESE,"https://api.weather.gov/icons/land/day/rain,20...",Slight Chance Light Rain
1,2024-12-13 09:00:00-08:00,True,42,F,0.26,"{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 89}",5 mph,ESE,"https://api.weather.gov/icons/land/day/rain,30...",Chance Light Rain
2,2024-12-13 10:00:00-08:00,True,43,F,0.30,"{'unitCode': 'wmoUnit:degC', 'value': 4.444444...","{'unitCode': 'wmoUnit:percent', 'value': 89}",5 mph,SSE,"https://api.weather.gov/icons/land/day/rain,30...",Chance Light Rain
3,2024-12-13 11:00:00-08:00,True,44,F,0.36,"{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 89}",5 mph,SSE,"https://api.weather.gov/icons/land/day/rain,40...",Chance Light Rain
4,2024-12-13 12:00:00-08:00,True,45,F,0.39,"{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 87}",5 mph,SSE,"https://api.weather.gov/icons/land/day/rain,40...",Chance Light Rain
...,...,...,...,...,...,...,...,...,...,...,...
151,2024-12-19 15:00:00-08:00,True,52,F,0.19,"{'unitCode': 'wmoUnit:degC', 'value': 7.777777...","{'unitCode': 'wmoUnit:percent', 'value': 82}",6 mph,ENE,"https://api.weather.gov/icons/land/day/rain,20...",Slight Chance Light Rain
152,2024-12-19 16:00:00-08:00,True,51,F,0.25,"{'unitCode': 'wmoUnit:degC', 'value': 7.777777...","{'unitCode': 'wmoUnit:percent', 'value': 82}",6 mph,NE,"https://api.weather.gov/icons/land/day/rain,30...",Chance Light Rain
153,2024-12-19 17:00:00-08:00,True,50,F,0.25,"{'unitCode': 'wmoUnit:degC', 'value': 7.222222...","{'unitCode': 'wmoUnit:percent', 'value': 82}",6 mph,NE,"https://api.weather.gov/icons/land/day/rain,30...",Chance Light Rain
154,2024-12-19 18:00:00-08:00,False,48,F,0.25,"{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 83}",6 mph,NE,"https://api.weather.gov/icons/land/night/rain,...",Chance Light Rain


In [7]:
forecastGrid = temp.copy()

In [ ]:
for k in forecastGrid['properties'].keys():
    print(k)


In [ ]:
forecastGrid['properties']['temperature']

In [5]:
data['properties']['forecastHourly']
temp = return_json(data['properties']['forecastHourly'])

In [ ]:
forecastHourly = temp.copy()
del forecastHourly['@context']
del forecastHourly['type']
del forecastHourly['geometry']
del forecastHourly['properties']['elevation']

forecastHourly['properties']

In [ ]:
df = pd.DataFrame(forecast['properties']['periods'])
df['startTime'] = pd.to_datetime(df['startTime'])
df['startTime'] = df['startTime'].apply(lambda x : x.strftime('%Y-%m-%d %I:%M:%S %p'))
#df = df.set_index('startTime')
df['windSpeed_min'] = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[0]))
df['windSpeed_max']  = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[-1]))
df.drop(['number', 'endTime', 'temperatureTrend', 'windSpeed'], axis = 1, inplace = True)
df['probabilityOfPrecipitation'] = df['probabilityOfPrecipitation'].apply( lambda x: x['value'] / 100 if x['value'] is not None else 0 )

df.head()

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot temperature on the primary y-axis
ax1.plot(df['startTime'], df['temperature'], color='tab:red', label='Temperature (°F)')
ax1.set_xlabel('Date Time')
ax1.set_ylabel('Temperature (°F)', color='tab:red')
min_temperature = df['temperature'].min() - 5  # adding buffer
max_temperature = df['temperature'].max() + 5  # adding buffer
ax1.set_ylim(min_temperature, max_temperature)
ax1.tick_params(axis='y', labelcolor='tab:red')

# Create a second y-axis to plot probability of precipitation
ax2 = ax1.twinx()
ax2.plot(df['startTime'], df['probabilityOfPrecipitation'], color='tab:blue', label='Probability of Precipitation', linestyle='--')
ax2.set_ylabel('Probability of Precipitation', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim((0,1))

# Title and show the plot
plt.title('Temperature and Probability of Precipitation')
fig.tight_layout()
plt.show()


In [ ]:
dt.fromisoformat(forecast['properties']['periods'][0]['startTime'])


In [ ]:
from datetime import datetime as dt
dt.fromisoformat(forecast['properties']['generatedAt'])

In [ ]:
data['properties']

In [ ]:
arcpy.management.Merge(
    inputs="tl_2024_us_coastline;tl_2024_us_internationalboundary",
    output=r"C:\Users\dviab\Desktop\iCloudDrive\Documents\Coding\shapefile\US boundary\US boundary.gdb\tl_2024_us_coastline_Merge1",
    field_mappings='NAME "NAME" true true false 100 Text 0 0,First,#,tl_2024_us_coastline,NAME,0,99;MTFCC "MTFCC" true true false 5 Text 0 0,First,#,tl_2024_us_coastline,MTFCC,0,4,tl_2024_us_internationalboundary,MTFCC,0,4;IBTYPE "IBTYPE" true true false 1 Text 0 0,First,#,tl_2024_us_internationalboundary,IBTYPE,0,254;IBSTATUS "IBSTATUS" true true false 1 Text 0 0,First,#,tl_2024_us_internationalboundary,IBSTATUS,0,254',
    add_source="NO_SOURCE_INFO"
)

In [3]:
import geocoder
g = geocoder.ip('me')
print(g.latlng)


[43.9826, -124.0998]


In [2]:
pip install geocoder

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

